In [ ]:
!pip install plotly pandas panel --quiet
!pip install kaleido --quiet


In [ ]:
!pip install jupyter_bokeh

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import panel as pn
from io import BytesIO
pn.extension('plotly')


In [ ]:
df = pd.read_csv("youth_unemployment_global.csv")
df.columns = df.columns.str.lower().str.strip()

# Detect unemployment column
unemp_col = next((col for col in df.columns if "unemploy" in col), None)
print("Detected unemployment column:", unemp_col)

# Add region/subregion
region_df = pd.read_csv(
    "https://raw.githubusercontent.com/datasets/country-codes/master/data/country-codes.csv"
)
region_df = region_df[["ISO3166-1-Alpha-3", "Region Name", "Sub-region Name"]]
region_df.columns = ["countrycode", "region", "subregion"]
df = df.merge(region_df, on="countrycode", how="left")


Detected unemployment column: youthunemployment


In [ ]:
# Country selection
country_select = pn.widgets.Select(
    name='Country',
    options=['Global Average'] + sorted(df['countrycode'].dropna().unique().tolist()),
    value='Global Average'
)

# Year range slider
year_slider = pn.widgets.IntRangeSlider(
    name='Year Range',
    start=int(df['year'].min()),
    end=int(df['year'].max()),
    value=(int(df['year'].min()), int(df['year'].max())),
    step=1
)

# Chart type
chart_type_select = pn.widgets.Select(
    name='Chart Type',
    options=['Line', 'Bar', 'Scatter', 'Box'],
    value='Line'
)

# Export buttons
export_csv_btn = pn.widgets.Button(name='Export CSV', button_type='success')
export_plot_btn = pn.widgets.Button(name='Export Plot', button_type='primary')

# Output area
plot_pane = pn.pane.Plotly(height=500)
status_text = pn.pane.Markdown("")


In [ ]:
def get_filtered_data(country, year_range):
    data = df[(df['year'] >= year_range[0]) & (df['year'] <= year_range[1])]
    if country != 'Global Average':
        data = data[data['countrycode'] == country]
    else:
        # Compute global average per year
        data = data.groupby('year')[unemp_col].mean().reset_index()
    return data


In [ ]:
def update_plot(event=None):
    data = get_filtered_data(country_select.value, year_slider.value)

    if chart_type_select.value == 'Line':
        fig = px.line(data, x='year', y=unemp_col, title=f"Youth Unemployment — {country_select.value}")
    elif chart_type_select.value == 'Bar':
        fig = px.bar(data, x='year', y=unemp_col, title=f"Youth Unemployment — {country_select.value}")
    elif chart_type_select.value == 'Scatter':
        fig = px.scatter(data, x='year', y=unemp_col, title=f"Youth Unemployment — {country_select.value}")
    else:  # Box plot by region
        fig = px.box(
            df[df['year'].between(year_slider.value[0], year_slider.value[1])],
            x='region',
            y=unemp_col,
            title='Regional Unemployment Distribution'
        )
    plot_pane.object = fig


In [ ]:
def export_csv(event):
    data = get_filtered_data(country_select.value, year_slider.value)
    buffer = BytesIO()
    data.to_csv(buffer, index=False)
    buffer.seek(0)
    with open("exported_data.csv", "wb") as f:
        f.write(buffer.read())
    status_text.object = "✔ CSV exported → exported_data.csv"


In [ ]:
def export_plot(event):
    data = get_filtered_data(country_select.value, year_slider.value)
    # Generate figure again
    if chart_type_select.value == 'Line':
        fig = px.line(data, x='year', y=unemp_col)
    elif chart_type_select.value == 'Bar':
        fig = px.bar(data, x='year', y=unemp_col)
    elif chart_type_select.value == 'Scatter':
        fig = px.scatter(data, x='year', y=unemp_col)
    else:
        fig = px.box(
            df[df['year'].between(year_slider.value[0], year_slider.value[1])],
            x='region',
            y=unemp_col
        )
    fig.write_image("exported_plot.png")
    status_text.object = "✔ Plot exported → exported_plot.png"


In [ ]:
country_select.param.watch(update_plot, 'value')
year_slider.param.watch(update_plot, 'value')
chart_type_select.param.watch(update_plot, 'value')
export_csv_btn.on_click(export_csv)
export_plot_btn.on_click(export_plot)


Watcher(inst=Button(button_type='primary', name='Export Plot'), cls=<class 'panel.widgets.button.Button'>, fn=<function export_plot at 0x793428fd1940>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [ ]:
dashboard = pn.Column(
    "# Interactive Youth Unemployment Dashboard",
    pn.Row(country_select, chart_type_select, year_slider),
    pn.Row(export_csv_btn, export_plot_btn, status_text),
    plot_pane,
    pn.pane.Markdown("""
### User Guide
- Select a **country** or 'Global Average'
- Adjust the **year range**
- Choose **chart type** (Line, Bar, Scatter, Box)
- Click **Export CSV** to download filtered data
- Click **Export Plot** to save the current chart as PNG
""")
)

dashboard.servable()



Column
    [0] Markdown(str)
    [1] Row
        [0] Select(name='Country', options=['Global Average', ...], value='Global Average')
        [1] Select(name='Chart Type', options=['Line', 'Bar', ...], value='Line')
        [2] IntRangeSlider(end=2024, name='Year Range', start=1960, value=(1960, 2024), value_end=2024, value_start=1960)
    [2] Row
        [0] Button(button_type='success', name='Export CSV')
        [1] Button(button_type='primary', name='Export Plot')
        [2] Markdown(str)
    [3] Plotly(None, height=500)
    [4] Markdown(str)